## My Classifier test score
- Binary Classifier for MNIST: 92.89%
- Multi Class Classifier for MNIST: 93.56%
- Multi Class Classifier for CIFAR10: 51.47%

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

c:\Users\jwhau\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Exercise 1-1
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html7

class MLP(nn.Module) :
    def __init__(self, in_dim, out_dim, inter_dim=28*28) :
        super().__init__()
        self.linear1 = nn.Linear(in_dim, inter_dim, bias=True)
        self.act1 = nn.Sigmoid()
        self.linear2 = nn.Linear(inter_dim, out_dim, bias=True)
        
    def forward(self, x) :
#         assert x.size()[2]*x.size()[3] == self.in_dim, "x.size()[2]*x.size()[3] should be {}".format(self.in_dim)
        z = self.linear1(x.view(-1, 28*28))
        z = self.act1(z)
        z = self.linear2(z)
        return z

In [3]:
STUDENT_model = MLP(28*28, 1)

In [4]:
# Please change "STUDENT_model" with your NN model class's instance name
total_params = sum(p.numel() for p in STUDENT_model.parameters())

print("Number of Parameters: ", total_params)


Number of Parameters:  616225


In [5]:
# Normalize data with mean=0.5, std=1.0
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))
])

# download path 정의
download_root = './MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [6]:
'''
Choose two numbers and prepare datasets with two numbers.
'''
nb1 = 3
nb2 = 8

train_dataset.data = torch.cat([train_dataset.data[(train_dataset.targets == nb1)],train_dataset.data[(train_dataset.targets == nb2)]],dim=0)
train_dataset.targets = torch.cat([train_dataset.targets[(train_dataset.targets == nb1)],train_dataset.targets[(train_dataset.targets == nb2)]],dim=0)

train_dataset.targets[(train_dataset.targets == nb1)] = 0
train_dataset.targets[(train_dataset.targets == nb2)] = 1

'''
Do the same thing for the test, valid dataset
'''
valid_dataset.data = torch.cat([valid_dataset.data[(valid_dataset.targets == nb1)],valid_dataset.data[(valid_dataset.targets == nb2)]],dim=0)
valid_dataset.targets = torch.cat([valid_dataset.targets[(valid_dataset.targets == nb1)],valid_dataset.targets[(valid_dataset.targets == nb2)]],dim=0)

valid_dataset.targets[(valid_dataset.targets == nb1)] = 0
valid_dataset.targets[(valid_dataset.targets == nb2)] = 1

test_dataset.data = torch.cat([test_dataset.data[(test_dataset.targets == nb1)],test_dataset.data[(test_dataset.targets == nb2)]],dim=0)
test_dataset.targets = torch.cat([test_dataset.targets[(test_dataset.targets == nb1)],test_dataset.targets[(test_dataset.targets == nb2)]],dim=0)

test_dataset.targets[(test_dataset.targets == nb1)] = 0
test_dataset.targets[(test_dataset.targets == nb2)] = 1

In [7]:
# option 값 정의
batch_size = 8
dim = 28*28
learning_rate = 1e-3

train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [8]:
my_loss = nn.BCELoss()

In [9]:
optimizer = optim.SGD(STUDENT_model.parameters(), learning_rate)

for epoch in range(12):
    STUDENT_model.train()
    epoch_loss = 0
    
    for i, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        output = STUDENT_model(data)
        
        output = output.reshape(-1)
        act = nn.Sigmoid()
        loss = my_loss(act(output), target.float())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f'Epoch: {epoch} | Train Loss: {epoch_loss / len(train_loader)}')

Epoch: 0 | Train Loss: 0.6707095100739611
Epoch: 1 | Train Loss: 0.6184385106544469
Epoch: 2 | Train Loss: 0.5572276433136498
Epoch: 3 | Train Loss: 0.4883371150302632
Epoch: 4 | Train Loss: 0.4217851060095712
Epoch: 5 | Train Loss: 0.36596553864203724
Epoch: 6 | Train Loss: 0.3225957646926191
Epoch: 7 | Train Loss: 0.2890395888777696
Epoch: 8 | Train Loss: 0.26301296242327493
Epoch: 9 | Train Loss: 0.24274342813005595
Epoch: 10 | Train Loss: 0.22644370502589622
Epoch: 11 | Train Loss: 0.21296768409208716


In [10]:
### Test for train_dataset
count = 0
for i in range(train_dataset.targets.size()[0]):
    image = train_dataset.data[i]
    label = train_dataset.targets[i]

    prediction = (STUDENT_model(image.float().view(-1, 28*28)) >= torch.FloatTensor([0.5]))
    if prediction == label:
        count += 1

print('total: {}, correctly matched: {}, accuracy: {:.2f}%\n'.format(train_dataset.targets.size()[0], count, count/train_dataset.targets.size()[0] * 100. ) )

total: 11982, correctly matched: 10914, accuracy: 91.09%



In [11]:
''' Test for test_dataset '''
count = 0
wrong_answer = []
STUDENT_model.eval()
for i in range(test_dataset.targets.size()[0]):
    image = test_dataset.data[i]
    label = test_dataset.targets[i]

    prediction = STUDENT_model.forward(image.float().view(-1, 28*28)) >= torch.FloatTensor([0.5])
    if prediction == label:
        count += 1
    else:
        wrong_answer += [i]

print('total: {}, correctly matched: {} accuracy: {:.2f}%\n'.format(test_dataset.targets.size()[0], count, count/test_dataset.targets.size()[0] * 100. ) )

total: 1984, correctly matched: 1839 accuracy: 92.69%

